Top-down approach (recursive)

In [1]:
from functools import wraps


def memo(func):
    cache = {}  # Stored subproblem solutions

    @wraps(func)  # Make wrap look like func
    def wrap(*args):  # The memoized wrapper
        if args not in cache:  # Not already computed?
            cache[args] = func(*args)  # Compute & cache the solution
        return cache[args]  # Return the cached solution

    return wrap


class ItemsKnapsack:
    def __init__(self, name, weight, value):
        self.name = name
        self.weight = weight
        self.value = value


@memo  # m is memoized (DECORATOR)
def memoized_knapsack(things, capacity):
    def m_val(num_objects, remaining_capacity):
        if num_objects == 0 or remaining_capacity == 0: return 0, 0, []
        index = num_objects - 1  # Object under consideration
        drop_val, drop_weight, drop_items = m_val(num_objects - 1, remaining_capacity)  # What if we drop the object?
        if things[index].weight > remaining_capacity: return drop_val, drop_weight, drop_items  # Too heavy: Must drop it
        value_obj, weight_obj, items_obj = m_val(num_objects - 1, remaining_capacity - things[index].weight)
        take_val = things[index].value + value_obj
        take_weight = things[index].weight + weight_obj
        take_items = items_obj + [things[index]]
        if drop_val > take_val:
            return drop_val, drop_weight, drop_items
        else:
            return take_val, take_weight, take_items

    return m_val(len(things), capacity)


# example (all those weigh 37)
bread = ItemsKnapsack("Lembas", 2, 6)
sword = ItemsKnapsack("Sting", 5, 8)
water_pouch = ItemsKnapsack("Water pouch", 4, 4)
coat = ItemsKnapsack("Elven cloak", 4, 6)
ring = ItemsKnapsack("The one ring", 1, 10)
ranged_weapon = ItemsKnapsack("Short bow", 3, 4)
tent = ItemsKnapsack("Tent", 5, 6)
tool = ItemsKnapsack("Tinderbox", 1, 1)
toiletries = ItemsKnapsack("Soap", 2, 1)
armour = ItemsKnapsack("Mithril shirt", 2, 9)
money = ItemsKnapsack("Money pouch", 2, 1)
map_middle_earth = ItemsKnapsack("Map", 1, 7)
bed = ItemsKnapsack("Bedroll", 3, 5)
rope = ItemsKnapsack("Elven rope", 2, 4)

items = [bread, sword, water_pouch, coat, ring, ranged_weapon, tent, tool, toiletries, armour, map_middle_earth, bed, rope]


result = memoized_knapsack(tuple(items), 5)
print("Max value:", result[0])
print("Corresponding total weight:", result[1])
print("Items:")
for item in result[2]:
    print(item.name)


Max value: 27
Corresponding total weight: 5
Items:
The one ring
Tinderbox
Mithril shirt
Map


Bottom-up approach (iterative)

In [2]:
class ItemsKnapsack:
    def __init__(self, name, weight, value):
        self.name = name
        self.weight = weight
        self.value = value


def knapsack(items, capacity):
    num_available_items = len(items)
    weight = [item.weight for item in items]
    value = [item.value for item in items]

    max_value_matrix = [[0] * (capacity + 1) for i in range(num_available_items + 1)]
    keep_drop_matrix = [[False] * (capacity + 1) for j in range(num_available_items + 1)]

    for k in range(1, num_available_items + 1):  # k first objects
        i = k - 1
        for r in range(1, capacity + 1):
            max_value_matrix[k][r] = drop = max_value_matrix[k - 1][r]
            if weight[i] > r:
                continue
            keep = value[i] + max_value_matrix[k - 1][r - weight[i]]
            max_value_matrix[k][r] = max(drop, keep)
            keep_drop_matrix[k][r] = keep > drop

    return max_value_matrix, keep_drop_matrix


def extract_items(items, P):
    k, remaining_cap, selected_items = len(items), len(P[0]) - 1, set()
    while k > 0 and remaining_cap > 0:
        i = k - 1
        if P[k][remaining_cap]:
            selected_items.add(items[i])
            remaining_cap -= items[i].weight
        k -= 1
    return selected_items


# Example usage:
bread = ItemsKnapsack("Lembas", 2, 6)
sword = ItemsKnapsack("Sting", 5, 8)
water_pouch = ItemsKnapsack("Water pouch", 4, 4)
coat = ItemsKnapsack("Elven cloak", 4, 6)
ring = ItemsKnapsack("The one ring", 1, 10)
ranged_weapon = ItemsKnapsack("Short bow", 3, 4)
tent = ItemsKnapsack("Tent", 5, 6)
tool = ItemsKnapsack("Tinderbox", 1, 1)
toiletries = ItemsKnapsack("Soap", 2, 1)
armour = ItemsKnapsack("Mithril shirt", 2, 9)
money = ItemsKnapsack("Money pouch", 2, 1)
map_middle_earth = ItemsKnapsack("Map", 1, 7)
bed = ItemsKnapsack("Bedroll", 3, 5)
rope = ItemsKnapsack("Elven rope", 2, 4)


items = [bread, sword, water_pouch, coat, ring, ranged_weapon, tent, tool, toiletries, armour, map_middle_earth, bed, rope]
knapsack_capacity = 5

m, P = knapsack(items, knapsack_capacity)
selected_items = extract_items(items, P)


max_value = m[len(items)][knapsack_capacity]
print(f"Maximum value: {max_value}")

print("Corresponding total weight:", knapsack_capacity)

print("Items:")
for item in selected_items:
    print(item.name)


# Print the max_value_matrix
print("Max Value Matrix:")
for row in m:
    print(row)

# Print the keep_drop_matrix
print("\nKeep Drop Matrix:")
for row in P:
    print(row)



Maximum value: 27
Corresponding total weight: 5
Items:
Mithril shirt
The one ring
Tinderbox
Map
Max Value Matrix:
[0, 0, 0, 0, 0, 0]
[0, 0, 6, 6, 6, 6]
[0, 0, 6, 6, 6, 8]
[0, 0, 6, 6, 6, 8]
[0, 0, 6, 6, 6, 8]
[0, 10, 10, 16, 16, 16]
[0, 10, 10, 16, 16, 16]
[0, 10, 10, 16, 16, 16]
[0, 10, 11, 16, 17, 17]
[0, 10, 11, 16, 17, 17]
[0, 10, 11, 19, 20, 25]
[0, 10, 17, 19, 26, 27]
[0, 10, 17, 19, 26, 27]
[0, 10, 17, 19, 26, 27]

Keep Drop Matrix:
[False, False, False, False, False, False]
[False, False, True, True, True, True]
[False, False, False, False, False, True]
[False, False, False, False, False, False]
[False, False, False, False, False, False]
[False, True, True, True, True, True]
[False, False, False, False, False, False]
[False, False, False, False, False, False]
[False, False, True, False, True, True]
[False, False, False, False, False, False]
[False, False, False, True, True, True]
[False, False, True, False, True, True]
[False, False, False, False, False, False]
[False, False, F